# Decoding Cellphone Signals with a Quantum Computer
This notebook demonstrates the use of a quantum computer in decoding transmissions used
in cellphone networks and compares the performance to the state-of-the-art classical 
alternatives.

1. [Modeling a Network](#Modeling-a-Network) section creates a model of a small radio network
2. [Decoding Transmissions: Classical Resources](#Decoding-Transmissions:-Classical-Resources) section uses 
    classical decoding techniques
3. [Decoding Transmissions: Quantum Resources](#Decoding-Transmissions:-Quantum-Resources) section demonstrates 
    using a quantum computer to decode transmissions.
4. [Supplementary Technical Material](#Supplementary-Technical-Material) section provides additional information 
    on some of the technical aspects of this notebook.

## A Hard (and Expensive) Problem

A paper [[1]](#1) presented in [SIGCOMM](https://www.sigcomm.org/) 2019 states, "For optimal performance, 
an amount of computation that increases at an exponential rate both with the number of users and with the 
data rate of each user is often required. The base station’s computational capacity is thus becoming one 
of the key limiting factors on wireless capacity." 

The world's ubiquitous cellphones require huge numbers of base stations to provide the cellular network; for 
example, in mid 2022, China's Ministry of Industry and Information Technology reported having deployed almost 
[2 million base stations](https://techblog.comsoc.org/2022/08/19/china-miit-claim-475m-5g-mobile-users-1-97m-5g-base-stations-at-end-of-july-2022/) 
for its 5G network alone. Operational costs are crucial to network providers and powering base stations 
[is challenging](https://techblog.comsoc.org/2020/08/07/5g-base-station-deployments-open-ran-competition-huge-5g-bs-power-problem/).  

The number of base stations required to serve a large city depends on the bandwidth such stations are 
capable of serving. Unfortunately, increases in cellphone numbers and noise quickly harden [[2]](#2) the 
problem of decoding transmissions. The graphic below shows the dependency of transmission errors on 
[SNRb](https://en.wikipedia.org/wiki/Signal-to-noise_ratio) (signal-to-noise ratio) and the ratio of 
cellphones to base stations ($\frac{Tx}{Rx}$). 

<img src="_static/problem_hardness_3d.png" width="500">

Two drivers of profitability for network providers are reducing power consumption and capturing 
any differences in processing capabilities.

## What is Coordinated Multipoint?

Very quickly after the deployment of the first generation of cellular networks it became clear that 
the naive model of pairing cellphones to a single nearby base station is inadequate. Many technologies 
have been developed to boost performance. 
[Cooperative MIMO](https://en.wikipedia.org/wiki/Cooperative_MIMO) (multiple-input multiple-output) 
effectively exploits the spatial domain of mobile fading channels to significantly improve performance. 
In one of its variations, Coordinated multipoint (CoMP), neighboring base stations share data and 
channel state information to enable them to coordinate downlink transmissions and jointly process 
received signals.  

## References

<a name="1">[1]</a> Minsung Kim, Davide Venturelli, and Kyle Jamieson. 
Leveraging quantum annealing for large MIMO processing in centralized radio access networks.
SIGCOMM '19: Proceedings of the ACM Special Interest Group on Data Communication, August 2019, Pages 241–255 
https://dl.acm.org/doi/10.1145/3341302.3342072

<a name="2">[2]</a> Toshiyuki Tanaka. 
A Statistical-Mechanics Approach to Large-System Analysis of CDMA Multiuser Detectors.
IEEE TRANSACTIONS ON INFORMATION THEORY, VOL. 48, NO. 11, NOVEMBER 2002


# Modeling a Network

TBD: explain the King's graph-based network model

The first code cell imports the needed funcions
NOTE: The code imports from "helpers", a folder colocated with this Jupyter Notebook.

In [ ]:
# General Python packages
import matplotlib.pyplot as plt
import numpy as np

# Ocean software packages
import dimod
from dwave.samplers import SimulatedAnnealingSampler
from dwave.system import DWaveSampler, FixedEmbeddingComposite

# Functions of this repository
from helpers.draw import draw_loop_comparison, draw_network
from helpers.filters import apply_filter, apply_filters, compare_signals, create_filter, create_filters, time_filter_instantiation
from helpers.general import loop_comparisons
from helpers.network import configure_network, create_channels, print_network_stats, simulate_signals

# Enable graphics inline
%matplotlib inline 

## Create a Network Graph

TBD: explain the King's graph-based network model



In [ ]:
network, _ = configure_network(network_size=5)

print_network_stats(network)
draw_network(network)

The next code cell plots the network graph, showing transmitters in red and receivers in green. 

## Create Channels

Between the transmitters and receivers there are channels etc

In [ ]:
channels, channel_power = create_channels(network)
print(f"Channels are represented by a {channels.shape[0]}x{channels.shape[1]} matrix.")

# Decoding Transmissions: Classical Resources

PLACEHOLDER TEXT: Processing time is crucial: a receiver’s physical layer must decode a transmission frame before the transmitter 
requires feedback about its decoding success or failure. For Wi-Fi networks, this translates to a hard requirement
of tens of microseconds. Even in newer cellphone networks such as 4G LTE, base stations must process transmissions 
with similar speed due to redundancy schemes. 

To meet processing-time restrictions, current networks mostly use linear filters, at the cost
of suboptimal performance.



## Create Filters

Standard linear filters used in most networks are the following:

* [Matched filter](https://en.wikipedia.org/wiki/Matched_filter)
* [Zero forcing](https://en.wikipedia.org/wiki/Zero-forcing_precoding) 
* [Minimum mean squared error (MMSE)](https://en.wikipedia.org/wiki/Minimum_mean_square_error)

Create all three of these filters for use in decoding a transmission.

In [ ]:
filters = create_filters(channels)
print(f"Created filters: {list(filters.keys())}.")

## Simulate Transmissions


In [ ]:
y, transmitted_symbols = simulate_signals(channels, channel_power)                 
print(f"First 10 transmitted symbols: {transmitted_symbols.flatten()[:10]}. \nFirst 10 received symbols: {y.flatten()[:10]}.")

## Decode Received Signals

In [ ]:
v = apply_filters(y, filters)
compare_signals(v, transmitted_symbols)

# Scaling Up

Processing time is crucial: a receiver’s physical layer must decode a transmission frame before the transmitter 
requires feedback about its decoding success or failure. For Wi-Fi networks, this translates to a hard requirement
of tens of microseconds. Even in newer cellphone networks such as 4G LTE, base stations must process transmissions 
with similar speed due to redundancy schemes. As a result, most current systems adopt
linear filters, accepting a drop in performance.

PLACEHOLDER TEXT: The function below compares ..... Times greater than half a second are highlighted in red. 

See more details in the {link to supplementary}

In [ ]:
time_filter_instantiation(network_size=[5, 10, 15])

# Decoding Transmissions: Quantum Resources
Some words here

In [ ]:
qpu = DWaveSampler(solver=dict(topology__type="pegasus"))

print(f"Selected {qpu.solver.name} with {len(qpu.nodelist)} qubits.")

## BQM Representation (Small-Town Problem)

### Create a Network, Filter, and Simulate Signals

In [ ]:
SNR = 10

network, embedding = configure_network(network_size=16, qpu=qpu)
print_network_stats(network)

channels, channel_power =  create_channels(network)

filter_mf = create_filter(channels, method='matched_filter')
y, transmitted_symbols = simulate_signals(channels, channel_power, SNRb=SNR)  

For comparison, decode this tranmission with the linear filter.

In [ ]:
v = apply_filter(filter_mf, y) 
compare_signals(v, transmitted_symbols)

### Create a BQM

In [ ]:
bqm = dimod.generators.mimo.spin_encoded_comp(network, 
                                              modulation = 'BPSK', 
                                              transmitted_symbols=transmitted_symbols, 
                                              F_distribution=('binary','real'), 
                                              F=channels,
                                              y=y)

print(f"BQM has {len(bqm)} variables with {len(bqm.quadratic)} quadratic interactions.")

### Decode Received Signals

Use the quantum computer to decode the tranmission.

In [ ]:
sampler = FixedEmbeddingComposite(qpu, embedding)

sampleset = sampler.sample(bqm, 
                           num_reads=30, 
                           annealing_time=200, 
                           chain_strength=-0.13*min(bqm.linear.values()),
                           label='Notebook - Coordinated Multipoint')

compare_signals(sampleset, transmitted_symbols)

## Big-City Problems

Problems become harder [[2]](#2) as a function of channel noise and base-station load. The graphic below shows the bit error rate (BER) as a function of SNRb and $\frac{Tx}{Rx}$ for a large network (`network_size=16`). Each data point is the median of ten problems solved by running simulated annealing for hundreds of times longer than the runtimes used in this notebook (around 10 seconds verus 30 milliseconds here).

<img src="_static/problem_hardness.png" width="800">

One sees, for example, that for SNRb of 5 and $\frac{Tx}{Rx}=1.5$, the expected BER approaches 10%. For problems with parameters above these BER lines, the BQM energy can be lowest (or the maximum likelihood greatest) for solutions that do not match the transmission.   

The next cell runs `runs` number of problems. If your Leap account has a limited quota, consider running fewer problems.  

In [ ]:
results = loop_comparisons(runs=10, qpu=qpu)
medians = {filter: np.median(val) for filter, val in results.items()}

for solver in medians:
    print(f'\t * Median success rate for {solver}: {medians[solver]}.')

draw_loop_comparison(results)

### Rough-Neighborhood Problems

You can configure and run for your own preferred values of SNR and $\frac{Tx}{Rx}$ ratio below.

In [ ]:
SNR = 4.5
tx_to_rx = 1.65


results = loop_comparisons(runs=3, qpu=qpu, snr=SNR, ratio=tx_to_rx)
medians = {filter: np.median(val) for filter, val in results.items()}

for solver in medians:
    print(f'\t * Median success rate for {solver}: {medians[solver]}.')

draw_loop_comparison(results, SNR=SNR, ratio=tx_to_rx)

# Supplementary Technical Material

This section provides more technical details on some of the topics demonstrated above.

## Performance of Matrix-Inverting Filters 

PLACEHOLDER TEXT: 

The following plot shows the decoding success rate and instantiation time for a range of network sizes. Each data point in the left plot is the median of ten problems with SNRb of 5 and $\frac{Tx}{Rx}=1.5$. Times in the right plot were measure on a 10-core i5 Intel processor operating at 1600 Mhz.  

<img src="_static/all_matched_filters_performance.png" width="800">

As noted in the [Decoding Transmissions: Classical Resources](#Decoding-Transmissions:-Classical-Resources) section, base stations in networks such as 4G LTE and Wi-Fi must process transmissions in tens of microseconds. Instantiation times that exceed a second are unacceptable. 

## Comparison to Simulated Annealing

PLACEHOLDER TEXT: Like linear filters that require matrix inversion, simulated annealing is also computationally demanding, requiring high power consumption. However, simulated annealing can produce high-quality results on the same order as the QPU.  

In [ ]:
results = loop_comparisons(runs=5, qpu=qpu, solvers=['SA'])
medians = {filter: np.median(val) for filter, val in results.items()}

for solver in medians:
    print(f'\t * Median success rate for {solver}: {medians[solver]}.')

draw_loop_comparison(results)

## Configuring Optimal QPU Parameters

PLACEHOLDER TEXT: How to select the non-default chain strength

The following plot shows BER and the fraction of broken chains for a range of chain strengths, calculated as `-cs_multiplier*min(bqm.linear.values())`, where `cs_multiplier` is the sought-after multiple of the most negative value of $h_i$, the linear coefficients of the BQM. Each data point is the median of twenty problems with SNRb of 5 and $\frac{Tx}{Rx}=1.5$ for a large network (`network_size=16`). 

<img src="_static/chain_strength.png" width="700">

## Minor Embedding

The graphic below shows how a small network graph (`problem_size=4`) is minor-embedded into the QPU working graph. 

Colored dots are qubits: note that pairs of identically-colored dots, such as the blue, green, and red pairs on the top row, represent a single node of the logical network. 

<img src="_static/embedding_lattice4.png" width="700">


## Advantage2: Boosting Performance 